In [1]:
import cv2
import numpy as np
import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers
from keras.layers import Input,Dense
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

In [2]:
from keras.models import model_from_json
json_file = open('model_final.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("model_final.h5")

In [178]:
img=cv2.imread("C:/Users/HP/Documents/hand written poly equation solver/test6.png",cv2.IMREAD_GRAYSCALE)
cv2.imshow("w",img)
cv2.waitKey(0)
cv2.destroyAllWindows()
if img is not None:
    img=~img
    _,thresh=cv2.threshold(img,127,255,cv2.THRESH_BINARY)
    ctrs,_=cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnt=sorted(ctrs,key=lambda ctr:cv2.boundingRect(ctr)[0])
    w=int(28)
    h=int(28)
    train_data=[]
    rects=[]
    for c in cnt:
        x,y,w,h=cv2.boundingRect(c)
        rect=[x,y,w,h]
        rects.append(rect)
    print("rects",rects)
    bool_rect=[]
    for r in rects:
        l=[]
        for rec in rects:
            flag=0
            if rec!=r:
                if r[0]<(rec[0]+rec[2]+10) and rec[0]<(r[0]+r[2]+10) and r[1]<(rec[1]+rec[3]+10) and rec[1]<(r[1]+r[3]+10):
                    flag=1
                l.append(flag)
            if rec==r:
                l.append(0)
        bool_rect.append(l)
    print('bools',bool_rect)
    dump_rect=[]
    for i in range(0,len(cnt)):
        for j in range(0,len(cnt)):
            if bool_rect[i][j]==1:
                area1=rects[i][2]*rects[i][3]
                area2=rects[j][2]*rects[j][3]
                if(area1==min(area1,area2)):
                    dump_rect.append(rects[i])
    print("dump_rects",dump_rect)
    final_rect=[i for i in rects if i not in dump_rect]
    print("final rects",final_rect)
    for r in final_rect:
        x=r[0]
        y=r[1]
        w=r[2]
        h=r[3]
        im_crop=thresh[y:y+h+10,x:x+w+10]
        im_resize=cv2.resize(im_crop,(28,28))
        cv2.imshow("work",im_resize)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        train_data.append(im_resize)

rects [[166, 232, 163, 119], [206, 305, 30, 37], [252, 240, 31, 43], [326, 139, 83, 71], [462, 262, 94, 17], [591, 214, 105, 98], [742, 184, 158, 107], [767, 245, 34, 32], [815, 199, 39, 29], [936, 170, 89, 126], [1073, 179, 97, 153]]
bools [[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
dump_rects [[206, 305, 30, 37], [252, 240, 31, 43], [767, 245, 34, 32], [815, 199, 39, 29]]
final rects [[166, 232, 163, 119], [326, 139, 83, 71], [462, 262, 94, 17], [591, 214, 105, 98], [742, 184, 158, 107], [936, 170, 89, 126], [1073, 179, 97, 153]]


In [179]:
s=""
for i in range(len(train_data)):
    train_data[i]=np.array(train_data[i])
    train_data[i]=train_data[i].reshape(1,28,28,1)
    result=np.argmax(loaded_model.predict(train_data[i]), axis=-1)
    if(result[0]==10):
        s=s+'-'
    if(result[0]==11):
        s=s+'+'
    if(result[0]==12):
        s=s+'X'
    if(result[0]==13):
        s=s+'*'
    if(result[0]==0):
        s=s+'0'
    if(result[0]==1):
        s=s+'1'
    if(result[0]==2):
        s=s+'2'
    if(result[0]==3):
        s=s+'3'
    if(result[0]==4):
        s=s+'4'
    if(result[0]==5):
        s=s+'5'
    if(result[0]==6):
        s=s+'6'
    if(result[0]==7):
        s=s+'7'
    if(result[0]==8):
        s=s+'8'
    if(result[0]==9):
        s=s+'9'
print(s)

X2-2X+4


In [180]:
def printEq(x):
    flag=0
    for i in range(len(x)):
        if flag==1:
            flag=0
            continue
        if i!=len(x)-1 and x[i]=='X'and x[i+1]!='-' and x[i+1]!='+' and x[i+1]!='*':
            print('X^'+str(x[i+1]), end=" ")
            flag=1
        else :
            print(x[i], end=" ")
        

In [181]:
def highdegree(x):
    Max=1
    for i in range(len(x)):
        if i!=len(x)-1 and x[i]=='X' and x[i+1]!='-' and x[i+1]!='+' and x[i+1]!='*':
            Max=max(Max,int(x[i+1]))
    return Max  

In [182]:
def solution(x):
    printEq(x)
    print("")
    degree=highdegree(x)
    print("degree is ",degree)
    eq=np.zeros(degree+1)
    ind=0
    for k in range(degree+1):
        #find constant term
        if k==0 :
            for i in range(len(x)):
                #x not infront , x not in back
                if i==0:
                    if x[i+1]!='X' and x[i].isdigit():
                        eq[degree-k]=x[i]
                        break
                elif i==len(x)-1:
                    if x[i-1]=='-' and x[i].isdigit():
                        eq[degree-k]=float(x[i])*-1
                        break
                    else:
                        if x[i-1]!='X' and x[i].isdigit():
                            eq[degree-k]=x[i]
                            break
                elif x[i+1]!='X' and x[i-1]!='X' and x[i].isdigit():
                    if x[i-1]=='-':
                        eq[degree-k]=float(x[i])*-1
                        break
                    else:
                        eq[degree-k]=x[i]
                        break
        if k==1:
            for i in range(len(x)):
                if i==len(x)-2:
                    if x[i+1]=='X':
                        eq[degree-k]=float(x[i])
                if(i<len(x)-2):    
                    if x[i+1]=='X' and x[i].isdigit():
                        if(not (x[i+2].isdigit())):
                            if x[i-1]=='-':
                                eq[degree-k]=float(x[i])*-1
                                break
                            else:
                                eq[degree-k]=x[i] 
                                break
                      
        if k>1:
            if x[0]=='X':
                eq[degree-k]=1
            for i in range(2,len(x)):
                if x[i].isdigit():
                    if int(x[i])==k and x[i-1]=='X':
                        if x[i-2].isdigit():
                            if i-3>=0 and x[i-3]=='-':
                                eq[degree-k]=float(x[i-2])*-1
                                break
                            else:
                                eq[degree-k]=x[i-2]
                                break
                        else:
                            eq[degree-k]=1
                            break
    #Print and return ans
    print(list(s))
    print(eq
          
          
    print(np.roots(eq))
    return np.roots(eq)

In [183]:
solution(list(s))

X^2 - 2 X + 4 
degree is  2
['X', '2', '-', '2', 'X', '+', '4']
[ 1. -2.  4.]
[1.+1.73205081j 1.-1.73205081j]


array([1.+1.73205081j, 1.-1.73205081j])